# Create the mask of the Beaufort Gyre Region used in the study

In [1]:
## standart libraries

import os,sys
import numpy as np

# xarray
import xarray as xr

# plot
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import matplotlib.pyplot as plt
from matplotlib.colors import Colormap
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
import matplotlib.cm as cm
import matplotlib.ticker as mticker
from matplotlib.colors import from_levels_and_colors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cmocean as ccm

# custom tools for plotting
import lib_SLXtools_IMHOTEP_2021 as slx

import matplotlib.style as style
style.use('tableau-colorblind10')

In [2]:
import multiprocessing.popen_spawn_posix
from dask.distributed import Client
c = Client()

/linkhome/rech/genlop01/uor98hu/.local/lib/python3.9/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37409 instead
  warnings.warn(


In [3]:
# read grid files
diri="/gpfswork/rech/cli/rcli002/eORCA025.L75/eORCA025.L75-I/"
tmask = xr.open_dataset(diri+'mesh_hgr.nc')['tmask'][0,0,:,:] # grille lat/lon
nav_lon = xr.open_dataset(diri+'mesh_hgr.nc')['nav_lon']
nav_lat = xr.open_dataset(diri+'mesh_hgr.nc')['nav_lat']
mbathy = xr.open_dataset(diri+'eORCA025.L75_domain_cfg_closed_seas_greenland.nc')['bathy_meter'].isel(t=0) # grille lat/lon

# land mask from model
m2plt      = tmask.where(tmask==0)
# default gridlines parameters
gridl=False
incrgridlon=30
incrgridlat=10

# vue N.Atl
loncentr=330 # before it was zero
latcentr=70
#vue polaire
loncentr=200 # before it was zero
latcentr=90

sath=2085831

#========= Plot settings ===============
# colormap
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = mcolors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

cmap = cm.RdYlBu_r
my_map1 = truncate_colormap(cmap)
colors = my_map1(np.linspace(0., 1, 50))
mycm = mcolors.LinearSegmentedColormap.from_list('my_colormap', colors)

lmin = -10.0
incr = 1
lmax = lmin*(-1)+0.1
# define the bins and normalize
bounds = np.arange(lmin,lmax,incr)
norm = mcolors.BoundaryNorm(bounds, mycm.N)

In [19]:
BGRmask = tmask.where((nav_lon<-130) & (nav_lon>-170) & (nav_lat<81) & (nav_lat>70)& (mbathy>300),0)

In [8]:
# saving the mask
ds = xr.Dataset(
            data_vars=dict(BGRmask=(["y", "x"], BGRmask.data)),
            attrs=dict(
                description="mask of the Beaufort Gyre region defined as tmask.where((nav_lon<-130) & (nav_lon>-160) & (nav_lat<80.5) & (nav_lat>70.5)& (mbathy>300),0)"))

ds.to_netcdf('/gpfswork/rech/cli/uor98hu/MYDATA/TRANSPORT_WP1/BGRmaskobs.nc', mode='w')
